In [ ]:
  import pandas as pd
import string
import re
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tqdm import tqdm
tqdm.pandas()
from nltk.sentiment import SentimentIntensityAnalyzer # VADER

In [ ]:
tweets = pd.read_csv('Bitcoin_tweets.csv')

<ipython-input-23-59bf8f8e8698>:1: DtypeWarning: Columns (5,6,7,12) have mixed types. Specify dtype option on import or set low_memory=False.
  tweets = pd.read_csv('Bitcoin_tweets.csv')


In [ ]:
tweets.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,DeSota Wilson,"Atlanta, GA","Biz Consultant, real estate, fintech, startups...",2009-04-26 20:05:09,8534.0,7605,4838,False,2021-02-10 23:59:04,Blue Ridge Bank shares halted by NYSE after #b...,['bitcoin'],Twitter Web App,False
1,CryptoND,NaN,😎 BITCOINLIVE is a Dutch platform aimed at inf...,2019-10-17 20:12:10,6769.0,1532,25483,False,2021-02-10 23:58:48,"😎 Today, that's this #Thursday, we will do a ""...","['Thursday', 'Btc', 'wallet', 'security']",Twitter for Android,False
2,Tdlmatias,"London, England","IM Academy : The best #forex, #SelfEducation, ...",2014-11-10 10:50:37,128.0,332,924,False,2021-02-10 23:54:48,"Guys evening, I have read this article about B...",NaN,Twitter Web App,False
3,Crypto is the future,NaN,I will post a lot of buying signals for BTC tr...,2019-09-28 16:48:12,625.0,129,14,False,2021-02-10 23:54:33,$BTC A big chance in a billion! Price: \487264...,"['Bitcoin', 'FX', 'BTC', 'crypto']",dlvr.it,False
4,Alex Kirchmaier 🇦🇹🇸🇪 #FactsSuperspreader,Europa,Co-founder @RENJERJerky | Forbes 30Under30 | I...,2016-02-03 13:15:55,1249.0,1472,10482,False,2021-02-10 23:54:06,This network is secured by 9 508 nodes as of t...,['BTC'],Twitter Web App,False


In [ ]:
tweets.user_verified.value_counts().index

Index([                                                                                                                                                                                                                                                                                                                     False,
                                                                                                                                                                                                                                                                                                                          'False',
                                                                                                                                                                                                                                                                                                                             True,
                               

In [ ]:
# Finding tweets where the total source count is 1

source_once = tweets.source.value_counts()[tweets.source.value_counts() == 1]

In [ ]:
# Removing tweets with source count 1, tweets with invalid dates

tweets = tweets[~tweets.date.isnull()]
tweets = tweets[~tweets.date.str.startswith("[")]
tweets = tweets[~tweets.source.isin(source_once)]

In [ ]:
# Converting date to datetime

tweets.date = pd.to_datetime(tweets.date, format = '%Y-%m-%d %H:%M:%S').dt.strftime('%Y-%m-%d')

In [ ]:
tweet_volumes = tweets.groupby('date')['text'].count()

In [ ]:
tweet_volumes.to_csv('tweet_volumes_bitcoin.csv')

In [ ]:
tweets_text = tweets.text

In [ ]:
# Stopwords

stop_words = set(stopwords.words('english'))

In [ ]:
# Emoji patterns to remove

emoji_pattern = re.compile("["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                       "]+", flags=re.UNICODE)

In [ ]:
# spaCy
nlp = English()
docs = nlp.pipe(tweets_text)

def clean_tweet_spacy(text):
    no_punc = [s[0].translate(str.maketrans('', '', string.punctuation)).lower() for s in text]
    whole = ''.join(no_punc).lower()
    cleaned = re.sub('@.*\s|http.*(\s|$)|#', '', whole).strip()
    
    #doc = nlp(cleaned)

    filtered = [] 

    token_list = []
    for token in doc:
        token_list.append(token.text)

    for word in token_list:
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False:
            filtered.append(word)
    cleaned = ' '.join(filtered)
    return emoji_pattern.sub(r'', cleaned)

In [ ]:
# NLTK
def clean_tweet_nltk(text):
    # removing punctuation and making lowercase
    no_punc = [s[0].translate(str.maketrans('', '', string.punctuation)).lower() for s in text]
    whole = ''.join(no_punc).lower()
    
    # removing @s, links, hashtag symbol (but keeping hashtag)
    cleaned = re.sub('@.*\s|http.*(\s|$)|#', '', whole).strip()
    
    word_tokens = word_tokenize(cleaned)
    
    # removing stopwords
    filtered = [w for w in word_tokens if not w.lower() in stop_words]
    cleaned = ' '.join(filtered)
    
    # removing emojis
    return emoji_pattern.sub(r'', cleaned)

In [ ]:
cleaned_df_spacy = tweets_text.progress_apply(clean_tweet_spacy)

In [ ]:
cleaned_df_nltk = tweets_text.progress_apply(clean_tweet_nltk)

100%|██████████| 2284913/2284913 [25:30<00:00, 1493.36it/s] 


In [ ]:
sentiment_df = pd.DataFrame(cleaned_df_nltk)

In [ ]:
sentiment_df

,text
0,blue ridge bank shares halted nyse bitcoin atm...
1,today thats thursday take 2 friend leowander...
2,guys evening read article btc would like share
3,btc big chance billion price 48726440 20210211...
4,network secured 9 508 nodes today soon biggest...
...,...
2284921,hiring sign free middlemen jobs jobsearching d...
2284922,stopped gg first 1r loss month still resistanc...
2284923,tylerscrypto hey broow trading going wan na do...
2284924,listenergy listenergy team working hard grow b...


In [ ]:
# creating sentiment analyzer pipe

sia = SentimentIntensityAnalyzer()

In [ ]:
# VADER method -- same as papers
# getting compound score (also has positive, negative, neutral scores -- compound is combination/total/overall)

def sentiment_analysis(x):
    return sia.polarity_scores(x)['compound']


In [ ]:
sentiment_df['sentiment'] = sentiment_df['text'].progress_apply(sentiment_analysis)

100%|██████████| 2284913/2284913 [11:21<00:00, 3352.41it/s]


In [ ]:
sentiment_df['date'] = tweets['date']

In [ ]:
sentiment_df.to_csv('sentiment_test.csv', index = False)